In [2]:
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
import time
from IPython.display import clear_output
# from joblib import Parallel, delayed
# from tqdm import tqdm
# import xlwings
# import datetime
# import mibian
# import swifter
# from joblib import Parallel, delayed

In [3]:
kite.ltp("NFO:BANKNIFTY24FEB37500PE")

{'NFO:BANKNIFTY24FEB37500PE': {'instrument_token': 18421506,
  'last_price': 1.1}}

In [4]:
def buy_sell_order(strike_price = None,ce_pe = None, price = None, trans_type = None,order_type = "MARKET",quantity = 30):
    print(strike_price, ce_pe, price,trans_type, order_type)
    order = kite.place_order(
        variety = kite.VARIETY_REGULAR,
        exchange = kite.EXCHANGE_NFO,
        tradingsymbol = f"BANKNIFTY24FEB{strike_price}{ce_pe}",
        transaction_type = trans_type,
        quantity = quantity,
        product = kite.PRODUCT_MIS,
        order_type = order_type,
        price=price,
        validity=kite.VALIDITY_DAY,
        disclosed_quantity=None,
        trigger_price=None,
        squareoff=None,
        stoploss=None,
        trailing_stoploss=None, 
        tag="TradeviaPython",
    )
    return order

In [ ]:
# Assuming 'kite' object is already defined and has necessary methods to fetch option prices
strike_price_ce = 47900
strike_price_pe = 45500
symbol_ce = f"NFO:BANKNIFTY24FEB{strike_price_ce}CE"
symbol_pe = f"NFO:BANKNIFTY24FEB{strike_price_pe}PE"

support_ce = 96
resistance_ce = 114
support_pe = 133
resistance_pe = 156

position_call  = 0 
position_put = 0 
call_side = True
put_side= True
breakout = True
position_put_breakout = 0
position_call_breakout = 0

while True:
    # Fetch current price for the call option
    price_ce = kite.ltp(symbol_ce)[symbol_ce]['last_price']
    price_pe = kite.ltp(symbol_pe)[symbol_pe]['last_price']
    print(f"CE: {price_ce}, PE: {price_pe}")
    # buy call
    if call_side:
        if price_ce <= support_ce*1.03 and position_call == 0 and position_put == 0:
            buy_sell_order(strike_price = strike_price_ce,ce_pe = "CE", price = None,trans_type="BUY")  
            position_call = 1
            print(f"Bought call at {price_ce}")

        # profit book
        if position_call == 1 and price_ce > resistance_ce*0.97:
            buy_sell_order(strike_price = strike_price_ce,ce_pe = "CE", price = None,trans_type="SELL") 
            position_call = 0
            print(f"Sold call at {price_ce}")

        # stop loss
        if position_call == 1 and price_ce <= support_ce*0.85:
            buy_sell_order(strike_price = strike_price_ce,ce_pe = "CE", price = None,trans_type="SELL") 
            position_call = 0
            print(f"Sold call at {price_ce}")
            # breakout
            if breakout:
                buy_sell_order(strike_price = strike_price_pe,ce_pe = "PE", price = None,trans_type="BUY") 
                position_put_breakout = 1
                breakout_price_pe = price_pe
                print(f"Bought put at breakout  {price_pe}")


        # breakout position SL hit
        if position_put_breakout == 1 and price_pe < breakout_price_pe*.85:
            buy_sell_order(strike_price = strike_price_pe,ce_pe = "PE", price = None,trans_type="SELL")
            break
        
        # breakout position profit book
        if position_put_breakout ==1 and price_pe > breakout_price_pe*1.50:
            buy_sell_order(strike_price = strike_price_pe,ce_pe = "PE", price = None,trans_type="SELL")
            break
            


    
    # buy put
    if put_side:
        if price_pe <= support_pe*1.03 and position_call == 0 and position_put == 0:
            buy_sell_order(strike_price = strike_price_pe,ce_pe = "PE", price = None,trans_type="BUY")  
            position_put = 1
            print(f"Bought put at {price_pe}")
        
        # profit book
        if position_put == 1 and price_pe > resistance_pe*0.97:
            buy_sell_order(strike_price = strike_price_pe,ce_pe = "PE", price = None,trans_type="SELL")
            position_put = 0
            print(f"Sold put at {price_pe}")

        
        # stop loss
        if position_put == 1 and price_pe <= support_pe*0.90:
            buy_sell_order(strike_price = strike_price_pe,ce_pe = "PE", price = None,trans_type="SELL") 
            position_put = 0
            print(f"Sold put at {price_pe}")
            # breakout
            if breakout:
                buy_sell_order(strike_price = strike_price_ce,ce_pe = "CE", price = None,trans_type="BUY")
                position_call_breakout = 1
                breakout_price_ce= price_ce

        # breakout position SL hit
        if position_put_breakout == 1 and price_ce < breakout_price_ce*.90:
            buy_sell_order(strike_price = strike_price_ce,ce_pe = "CE", price = None,trans_type="SELL")
            break
        
        # breakout position profit book
        if position_put_breakout ==1 and price_ce > breakout_price_ce*1.50:
            buy_sell_order(strike_price = strike_price_ce,ce_pe = "CE", price = None,trans_type="SELL")
            break
                           
    

    time.sleep(5)
        

In [4]:
# Assuming 'kite' object is already defined and has necessary methods to fetch option prices
strike_price = 21950
symbol_ce = f"NFO:NIFTY24222{strike_price}CE"
symbol_pe = f"NFO:NIFTY24222{strike_price}PE"
support = 25
resistance = 41

position_call  = 0 
position_put = 0 
price = None  # Placeholder for the option premium price

while True:
    # Fetch current price for the call option
    price_ce = kite.ltp(symbol_ce)[symbol_ce]['last_price']
    price_pe = kite.ltp(symbol_pe)[symbol_pe]['last_price']
    print(f"CE: {price_ce}, PE: {price_pe}")

    break
    # Buy call if price reached near support and no position is open
    if price_ce <= support*.99 and price_ce >= support*1.01 and position_call == 0 and position_put == 0:
        buy_sell_order(strike_price = strike_price,ce_pe = "CE", price = None,trans_type="BUY")  
        position_call = 1
        print(f"Bought call at {price_ce}")
    
    # Sell call if price reached near resistance and call position is open
    elif price_ce >= resistance and position_call == 1:
        buy_sell_order(strike_price = strike_price,ce_pe = "CE", price = None,trans_type="SELL") 
        position_call = 0
        print(f"Sold call at {price_ce}")
    
    # Sell call and buy put if call position is open and price goes below support by 10%
    elif position_call == 1 and price_ce < (support * 0.90):
        buy_sell_order(strike_price = strike_price,ce_pe = "CE", price = None,trans_type="SELL") 
        time.sleep(5)
        buy_sell_order(strike_price = strike_price,ce_pe = "PE", price = None, trans_type="BUY")
        position_call = 0  
        position_put = 1
        print(f"Switched to put at {price_pe}")
    
    # Buy call if price crosses the resistance by 5 points and no position is open
    elif price_ce > (resistance + 5) and position_call == 0:
        buy_sell_order(strike_price = strike_price,ce_pe = "CE", price = None,trans_type="BUY") 
        position_call = 1
        print(f"Bought call at {price_ce}")
    
    # time.sleep(5)  # Adjust the sleep time as needed
    # clear_output(wait=True)
    break

CE: 35.3, PE: 34.05


In [5]:
inst = pd.DataFrame(kite.instruments("NFO"))
inst_nse = pd.DataFrame(kite.instruments("NSE"))

In [6]:
inst[(inst.name == 'BANKNIFTY')]

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
3,13368322,52220,BANKNIFTY24APRFUT,BANKNIFTY,0.0,2024-04-25,0.0,0.05,15,FUT,NFO-FUT,NFO
4,18288642,71440,BANKNIFTY24FEBFUT,BANKNIFTY,0.0,2024-02-29,0.0,0.05,15,FUT,NFO-FUT,NFO
5,9372418,36611,BANKNIFTY24MARFUT,BANKNIFTY,0.0,2024-03-28,0.0,0.05,15,FUT,NFO-FUT,NFO
2104,18421250,71958,BANKNIFTY24FEB37500CE,BANKNIFTY,0.0,2024-02-29,37500.0,0.05,15,CE,NFO-OPT,NFO
2105,18421506,71959,BANKNIFTY24FEB37500PE,BANKNIFTY,0.0,2024-02-29,37500.0,0.05,15,PE,NFO-OPT,NFO
...,...,...,...,...,...,...,...,...,...,...,...,...
3575,17170434,67072,BANKNIFTY24DEC52500PE,BANKNIFTY,0.0,2024-12-26,52500.0,0.05,15,PE,NFO-OPT,NFO
3576,17170690,67073,BANKNIFTY24DEC54000CE,BANKNIFTY,0.0,2024-12-26,54000.0,0.05,15,CE,NFO-OPT,NFO
3577,17170946,67074,BANKNIFTY24DEC54000PE,BANKNIFTY,0.0,2024-12-26,54000.0,0.05,15,PE,NFO-OPT,NFO
3578,17171202,67075,BANKNIFTY24DEC55500CE,BANKNIFTY,0.0,2024-12-26,55500.0,0.05,15,CE,NFO-OPT,NFO


In [11]:
expiry_sp = 'NIFTY24222'
inst = pd.DataFrame(kite.instruments("NFO"))

expiry_march  = inst[(inst.name == 'NIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
# expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [12]:
expiry_march

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
773,9091074,35512,NIFTY2422219500CE,NIFTY,0.0,2024-02-22,19500.0,0.05,50,CE,NFO-OPT,NFO
774,9091330,35513,NIFTY2422219500PE,NIFTY,0.0,2024-02-22,19500.0,0.05,50,PE,NFO-OPT,NFO
775,9092098,35516,NIFTY2422219550CE,NIFTY,0.0,2024-02-22,19550.0,0.05,50,CE,NFO-OPT,NFO
776,9092354,35517,NIFTY2422219550PE,NIFTY,0.0,2024-02-22,19550.0,0.05,50,PE,NFO-OPT,NFO
777,9093634,35522,NIFTY2422219600CE,NIFTY,0.0,2024-02-22,19600.0,0.05,50,CE,NFO-OPT,NFO
...,...,...,...,...,...,...,...,...,...,...,...,...
948,10363906,40484,NIFTY2422223850PE,NIFTY,0.0,2024-02-22,23850.0,0.05,50,PE,NFO-OPT,NFO
949,15559170,60778,NIFTY2422223900CE,NIFTY,0.0,2024-02-22,23900.0,0.05,50,CE,NFO-OPT,NFO
950,15560194,60782,NIFTY2422223900PE,NIFTY,0.0,2024-02-22,23900.0,0.05,50,PE,NFO-OPT,NFO
951,15560450,60783,NIFTY2422223950CE,NIFTY,0.0,2024-02-22,23950.0,0.05,50,CE,NFO-OPT,NFO


In [4]:
# nifty_token = 256265 # NIFTY 50	
# expiry_day = 'NIFTY24215'

# expiry_march = (
#     inst[(inst.name == "NIFTY") & (inst.tradingsymbol.str.contains(expiry_day))]
#     # .query("(strike >= @lower_price )&( strike <= @upper_price)")
#     # .assign(strike_type=lambda x: x["strike"].astype(int).astype(str) + x["instrument_type"])
# )
# df = pd.DataFrame(kite.historical_data(nifty_token, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))

In [ ]:
kite.ltp

In [ ]:
# price consolidation i
from IPython.display import clear_output
import time
strike_price = 42900
position = 0
market  = False

lower_side = 42811
upper_side = 42911

target_lower = 49000
target_upper = 42488

sl_lower = 42228
sl_upper = 42299


if market:
    order_type  = kite.ORDER_TYPE_MARKET
else:
    order_type  = kite.ORDER_TYPE_LIMIT

symbol_ce = f"NFO:BANKNIFTY23504{strike_price}CE"
symbol_pe = f"NFO:BANKNIFTY23504{strike_price}PE"
# price = 385
while(True):
    bn_index = kite.ltp('NSE:NIFTY BANK')['NSE:NIFTY BANK']['last_price']
    op_begin_ce = kite.ltp(symbol_ce)[symbol_ce]['last_price']
    op_begin_pe = kite.ltp(symbol_pe)[symbol_pe]['last_price']
    print(bn_index,op_begin_ce,op_begin_pe)
    # buy 
    if bn_index > upper_side and position == 0:
        print('buy_up', bn_index) 
        trans_type = kite.TRANSACTION_TYPE_BUY
        op_begin_ce -= 5
        buy_sell_order(strike_price = strike_price,price =op_begin_ce, ce_pe = 'CE', trans_type = trans_type,order_type =order_type)
        position = 1 
        
    elif bn_index < lower_side and position == 0:
        print("buy_down",bn_index)
        trans_type = kite.TRANSACTION_TYPE_BUY
        op_begin_pe -= 5
        buy_sell_order(strike_price = strike_price,ce_pe = 'PE', trans_type = trans_type,order_type = order_type)
        position = 1
    
    # if position == 1 and (bn_index > target_upper or bn_index < sl_upper): 
    #     trans_type = kite.TRANSACTION_TYPE_SELL
    #     order_type  = kite.ORDER_TYPE_MARKET
    #     buy_sell_order(strike_price = strike_price,ce_pe = 'CE',price = None, trans_type = trans_type,order_type = order_type)
    #     break
    # elif position == 1 and (bn_index < target_lower or bn_index > sl_lower):
    #     trans_type = kite.TRANSACTION_TYPE_SELL
    #     buy_sell_order(strike_price = strike_price,ce_pe = 'PE',price = None, trans_type = trans_type,order_type = order_type)
    #     break
        
    time.sleep(5)
    clear_output(wait=True)
    # break

    